# Introduccion a Gradio

In [2]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from langchain_ollama.llms import OllamaLLM

In [3]:
import gradio as gr

In [4]:
system_message = "You are a helpful assistant"

In [31]:
def message_ollama(prompt, model_name):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    return OllamaLLM(model=model_name).invoke(messages)

In [ ]:
prompt = "What is Gradio?"
qwen_05b = "qwen2.5-coder:0.5b"
message_ollama(prompt, qwen_05b)

## Ejemplo de User Interface

In [8]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [ ]:
shout("hello")

In [ ]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

In [ ]:
# Añadir share=True significa que puede ser accedido públicamente
# NOTA: Algunos antivírus y firewall corporativos pueden no tolerar usar share=True. Si estás en un entorno de trabajo o en una red de trabajo, sugiero desactivar esta prueba.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
# Agregar inbrowser=True abre una ventana de navegador automaticamente

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

## Forzar 'Dark Mode'

Gradio se muestra en modo claro o oscuro dependiendo del conjunto de configuraciones del navegador y la
computadora. Hay una forma de forzar Gradio a mostrar el modo oscuro, pero Gradio recomienda no hacer esto ya que
debería ser una preferencia de accesibilidad ( especialmente para los usuarios). Sin embargo, para hacerlo siga estos pasos:

In [ ]:
# Define esta variable y luego pasa "force_dark_mode" al crear la interfaz.

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

In [ ]:
# Inputs y Outputs

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# Cambiar la funcion de demo por la que hace el llamado a nuestro LLM
model_value = gr.Text(label="LLM Model", value=qwen_05b)
view = gr.Interface(
    fn=message_ollama,
    inputs=[gr.Textbox(label="Your message:", lines=6), model_value],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never"
)
view.launch()

In [ ]:
# Cambiemos un poco el estilo de respuesta para obtenerlo como Markdown

system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_ollama,
    inputs=[gr.Textbox(label="Your message:"), model_value],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [43]:
# Ahora modifiquemos el llamado al LLM para que devuela la respuesta en modo de 'stream'
def stream_ollama(prompt, model_name):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    result = ""
    for chunk in OllamaLLM(model=model_name).stream(messages):
        result += chunk or ""
        yield result

In [ ]:
view = gr.Interface(
    fn=stream_ollama,
    inputs=[gr.Textbox(label="Your message:"), model_value],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [51]:
def stream_multi_model(prompt, model):
    if model=="qwen2.5-coder:0.5b":
        result = stream_ollama(prompt, "qwen2.5-coder:0.5b")
    elif model=="llama3.2":
        result = stream_ollama(prompt, "llama3.2")
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_multi_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["qwen2.5-coder:0.5b", "llama3.2"], label="Select model", value="qwen2.5-coder:0.5b")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

# Building a company brochure generator

Now you know how - it's simple!

In [ ]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
# With massive thanks to Bill G. who noticed that a prior version of this had a bug! Now fixed.

system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [ ]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()